# Stigma against Opioid Use Disorder varies by Personal Use status

```{margin} 
**To follow the full analysis, click through the hidden analysis code below**
```

In [ ]:
# import packages
import os
import json
from pathlib import Path
import pandas as pd
import numpy as np
import pyreadstat
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import bootstrap
import plotly.express as px

pd.set_option('mode.chained_assignment', None)

In [ ]:
# inputs
STATE_ABBREVIATIONS = "state_abbrev_mappings.json"
DATA_FILE = (
    "P:/3645/Common/Protocol 2 Custom Survey/"
    "Analysis/Data File/"
    "3645_JCOIN_HEAL Initiative 2021_NORC_Jan2022_1.sav"
)

In [ ]:
# import data and metadata (data dictionaries)
df, meta = pyreadstat.read_sav(DATA_FILE,apply_value_formats=True)


In [ ]:
df.head()

In [ ]:
# narrow down the dataset to only variables that are collected at each of the time-points

# standardize column names across datasets and metadatasets
for df in [df]:
    df.columns = df.columns.str.lower()

In [ ]:
vars_of_interest = ['p_over','weight1','weight2','stigma_scale_score','expanded_10item_stigma','state','age4','racethnicity','educ5','personaluse_ever','familyuse_ever','personalcrimjust_ever','familycrimjust_ever']
categorical_vars = ['p_over','state','age4','racethnicity','educ5',
    'personaluse_ever','familyuse_ever',
    'personalcrimjust_ever','familycrimjust_ever']

In [ ]:
# narrow down the dataset to only a few interesting (and relatively clean, straightforward variables) - check for missingness and impute to fill in missing
sub_df_1 = df[vars_of_interest]

In [ ]:
sub_df_1.head()

## Missing values and counts

In [ ]:
# check for missing I

# check if missing values
print("missing values (.isnull): ")
print(sub_df_1.isnull().sum())
print()
# check if missing values
print("missing values (.isna): ")
print(sub_df_1.isna().sum())
print()
# get all var types
#sub_df_1.info()
print("var info (.info): ")
print(sub_df_1.info())
print()
# summary of numeric vars (weight and stigma_scale_score)
print("summary of numeric vars (.describe): ")
print(sub_df_1.describe())



In [ ]:
# check for missing II
# summary of cat vars
for v in categorical_vars:
    print(f"summary of cat var {v}: ")
    print(sub_df_1[v].value_counts(dropna=False))


In [ ]:
# convert familycrimjust_ever var from 1/0 to yes/no to be in line with other covar format

#df.replace({0: 10, 1: 100},inplace=True)
sub_df_1.familycrimjust_ever.replace({0:"No",1:"Yes"},inplace=True)
print(sub_df_1.familycrimjust_ever.value_counts(dropna=False))


In [ ]:
# familyuse_ever var was coded as yes/' no' (space before 'no') instead of yes/no; convert it to be yes/no to be in line with other covar format

print(sub_df_1.familyuse_ever.value_counts(dropna=False))
sub_df_1.familyuse_ever.replace({" No":"No"},inplace=True)
print(sub_df_1.familyuse_ever.value_counts(dropna=False))


In [ ]:
# personalcrimjust_ever was coded as Yes, ever arrested or incarcerated/No, never arrested or incarcerated; convert it to be yes/no to be in line with other covar format

print(sub_df_1.personalcrimjust_ever.value_counts(dropna=False))
#sub_df_1.personalcrimjust_ever.replace(regex = {r'^Yes.$': "Yes", r'^No.$': "No"}, inplace=True)
sub_df_1.personalcrimjust_ever.replace({"Yes, ever arrested or incarcerated":"Yes", "No, never arrested or incarcerated":"No"},inplace=True)
print(sub_df_1.personalcrimjust_ever.value_counts(dropna=False))



In [ ]:
# impute missing stigma scale score vals with median per timepoint, impute missing personaluse_ever with mode, "No"

# impute missing stigma scale score values as the median score by survey time-point
#sub_df_1['stigma_scale_score'].fillna(sub_df_1.groupby('time-point')['stigma_scale_score'].transform('median'),inplace=True)
sub_df_1['stigma_scale_score'].fillna(sub_df_1['stigma_scale_score'].median(),inplace=True)
sub_df_1['expanded_10item_stigma'].fillna(sub_df_1['expanded_10item_stigma'].median(),inplace=True)
print(sub_df_1.isnull().sum())

# replace missing values of personaluse_ever with mode value of 'No'
sub_df_1.personaluse_ever.fillna('No',inplace=True)
print(sub_df_1.isnull().sum())

sub_df_1.familyuse_ever.fillna('No',inplace=True)
print(sub_df_1.isnull().sum())

sub_df_1.personalcrimjust_ever.fillna('No',inplace=True)
print(sub_df_1.isnull().sum())

sub_df_1.familycrimjust_ever.fillna('No',inplace=True)
print(sub_df_1.isnull().sum())



In [ ]:
sub_df_1.head()

In [ ]:
# get all var types
#sub_df_1.info()
print("var info: ")
print(sub_df_1.info())

# confirm missing values corrected
print("missing values: ")
print(sub_df_1.isnull().sum())

# confirm missing values corrected
print("missing values: ")
print(sub_df_1.isna().sum())

## State level weights

In [ ]:
# https://pythonfix.com/code/us-states-abbrev.py/

# state name to two letter code dictionary
us_state_to_abbrev = json.loads(Path(STATE_ABBREVIATIONS).read_text())


In [ ]:
# add df column with state 2 letter code

state_cd = sub_df_1.state.replace(us_state_to_abbrev)
state_cd

sub_df_1.insert(6,"state_cd",state_cd,True)
sub_df_1.head()


In [ ]:
# define function to calculate weighted mean of across the full population for survey vars - necessary to properly weight in order to retain the validity of nationally representative survey sampling strategy 
# https://stackoverflow.com/questions/32771520/how-to-use-a-weighted-mean-estimator-in-seaborn-factor-plot-incl-bootstrapping

# sum of weights will not be equal to count of individuals when we look at sub-groups of the full population; 
# will have to formally calculate weighted average
def weighted_mean(x, **kws):
    val, weight = map(np.asarray, zip(*x))
    return (val * weight).sum() / weight.sum()

#sub_df_1["score_and_weight"] = list(zip(sub_df_1.stigma_scale_score, sub_df_1.weight))


In [ ]:
# https://github.com/mwaskom/seaborn/issues/722

def weighted_mean_2(x, **kws):
    return np.sum(np.real(x) * np.imag(x)) / np.sum(np.imag(x))


sub_df_1["score6_and_weight2"] = [ v + w*1j for v,w in zip(sub_df_1.stigma_scale_score, sub_df_1.weight2)]
sub_df_1["score10_and_weight2"] = [ v + w*1j for v,w in zip(sub_df_1.expanded_10item_stigma, sub_df_1.weight2)]


In [ ]:
def wavg(group, avg_name, weight_name):
    """ http://stackoverflow.com/questions/10951341/pandas-dataframe-aggregate-function-using-multiple-columns
    In rare instance, we may not have weights, so just return the mean. Customize this if your business case
    should return otherwise.
    """
    d = group[avg_name]
    w = group[weight_name]
    try:
        return (d * w).sum() / w.sum()
    except ZeroDivisionError:
        return d.mean()

In [ ]:
# national estimate - method 1
print(weighted_mean_2(sub_df_1["score6_and_weight2"]))
print(weighted_mean_2(sub_df_1["score10_and_weight2"]))

# national estimate - method 2
print(wavg(sub_df_1,"stigma_scale_score","weight2"))
print(wavg(sub_df_1,"expanded_10item_stigma","weight2"))


In [ ]:
# state-level estimate - method 2 - all states

print(sub_df_1.groupby("state").apply(wavg,"stigma_scale_score","weight2"))
print(sub_df_1.groupby("state").apply(wavg,"expanded_10item_stigma","weight2"))


In [ ]:
# state-level estimate - method 2 - only oversampled states

# create new df with only state oversampled entries
sub_df_2 = sub_df_1.copy()
sub_df_2.state.cat.remove_unused_categories(inplace = True)

# use new sub_df_2 to get state level estimates
print(sub_df_2.groupby("state").apply(wavg,"stigma_scale_score","weight2"))
print(sub_df_2.groupby("state").apply(wavg,"expanded_10item_stigma","weight2"))



Get count of obs in oversampled states but not part of oversampled population:

In [ ]:
# \ is line continuation
oversampled_states = sub_df_1\
    [sub_df_1.p_over == "AS oversample"]\
    ['state']\
    .cat.remove_unused_categories()\
    .cat.categories

In [ ]:
# oversample pop
oversampled_pop_counts = (
    sub_df_1
    [lambda df:(df.p_over == "AS oversample") & (df['state'].isin(oversampled_states))]
    ['state']
    .value_counts(dropna=False)
)
print("TOTAL N:")
print(oversampled_pop_counts.sum())
print("N per State:")
print(
    oversampled_pop_counts
)

In [ ]:
#oversampled states but not oversampled pop
print(
    sub_df_1
    [lambda df:(df.p_over != "AS oversample") & (df['state'].isin(oversampled_states))]
    ['state']
    .value_counts()
)

In [ ]:
# state-level estimate - method 2 - only oversampled states 
isin_oversampled_state = sub_df_1["state"].isin(oversampled_states)
print(sub_df_1.loc[isin_oversampled_state,:].groupby("state", observed = True).apply(wavg,"stigma_scale_score","weight2"))
print(sub_df_1.loc[isin_oversampled_state,:].groupby("state", observed = True).apply(wavg,"expanded_10item_stigma","weight2"))

state_lvl_est_6 = sub_df_1.loc[isin_oversampled_state,:].groupby("state", observed = True).apply(wavg,"stigma_scale_score","weight2")
state_lvl_est_10 = sub_df_1.loc[isin_oversampled_state,:].groupby("state", observed = True).apply(wavg,"expanded_10item_stigma","weight2")


In [ ]:
state_lvl_est_6 = sub_df_1[sub_df_1.p_over == "AS oversample"].groupby("state", observed = True).apply(wavg,"stigma_scale_score","weight2")

state_lvl_est_6 = pd.DataFrame(state_lvl_est_6).reset_index().rename(columns={0:"stigma_scale_score"}).sort_values(by=["stigma_scale_score"],ascending=False)
state_lvl_est_6.head(10)

# add df column with state 2 letter code
def add_state_cd_to_df(df,state_name_col,insert_col,state_cd_col_name="state_cd"):
    # for df column with state full name, 
    # add df column with state 2 letter code
    # at desired location

    state_cd = df[state_name_col].replace(us_state_to_abbrev)
    state_cd

    # for now, leave in true to allow duplicates; should think about changing this though
    df.insert(insert_col,state_cd_col_name,state_cd,True)
    return df

state_lvl_est_6 = add_state_cd_to_df(df=state_lvl_est_6, state_name_col="state", insert_col=1)
state_lvl_est_6.head(10)    

In [ ]:
# quick map of state level estimates

fig = px.choropleth(state_lvl_est_6,
    locations="state_cd",
    locationmode="USA-states",
    scope="usa",
    color="stigma_scale_score",
    color_continuous_scale="Viridis_r")

fig.show()


In [ ]:
# iterating over a grouped df: http://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html#:~:text=the%20passed%20key.-,Iterating%20through%20groups,-%23

grouped_by_state = sub_df_2.groupby("state")

for name, group in grouped_by_state:
    print(name)
    print(group.weight2)

# https://allendowney.github.io/ElementsOfDataScience/12_bootstrap.html#weighted-bootstrapping

In [ ]:
from scipy.stats import bootstrap

# get a numpy array of stigma score values per state - need this to feed into scipy bootstrap function
state_stigma_value_arrays = sub_df_2.stigma_scale_score.groupby(sub_df_2.state).apply(np.array).values
print(state_stigma_value_arrays.ndim)
print(state_stigma_value_arrays.shape)
#print(state_stigma_value_arrays[5,])
print(state_stigma_value_arrays[14,].ndim)
print(state_stigma_value_arrays[14,].shape)

# get a numpy array of corresponding weight2 per state - need this to feed into scipy bootstrap function
state_stigma_value_arrays = sub_df_2.stigma_scale_score.groupby(sub_df_2.state).apply(np.array).values
print(state_stigma_value_arrays.ndim)
print(state_stigma_value_arrays.shape)
#print(state_stigma_value_arrays[5,])
print(state_stigma_value_arrays[14,].ndim)
print(state_stigma_value_arrays[14,].shape)




In [ ]:
# https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.bootstrap.html
n_trials = 5000
rng = np.random.default_rng()
from scipy.stats import norm
dist = norm(loc=2, scale=4) 
data = (dist.rvs(size=(n_trials, 100), random_state=rng),)
#res = bootstrap(data, np.std, axis=-1, confidence_level=0.9, n_resamples=1000, random_state=rng)
#ci_l, ci_u = res.confidence_interval
data

In [ ]:
# get all var types
#sub_df_1.info()
print("var info: ")
print(sub_df_1.info())

sub_df_1.head()

In [ ]:
ax = sns.histplot(x="stigma_scale_score", data=sub_df_1, binwidth=0.5)

for i in ax.containers:
    ax.bar_label(i,)

In [ ]:
ax = sns.histplot(x="stigma_scale_score", data=sub_df_1, binwidth=0.5, weights="weight2")

for i in ax.containers:
    ax.bar_label(i,)

In [ ]:
ax = sns.histplot(x="stigma_scale_score", data=sub_df_1, binwidth=0.2, weights="weight2", hue="personaluse_ever", alpha = 0.6, stat="percent", common_norm=False, multiple="layer")
# setting common_norm == False gives percent within each group


#for i in ax.containers:
#    ax.bar_label(i,)


In [ ]:
ax = sns.histplot(x="stigma_scale_score", data=sub_df_1, binwidth=0.2, weights="weight2", hue="familyuse_ever", alpha = 0.6, stat="percent", common_norm=False, multiple="layer")
# setting common_norm == False gives percent within each group


In [ ]:
fig, ax = plt.subplots()

sns.barplot(ax=ax, x="state", y="score6_and_weight2", data=sub_df_1, estimator=weighted_mean_2, orient='v')
#ax.set_ylim(2, 3.55) 

plt.draw()
ax.set_xticks(ax.get_xticks())
ax.set_xticklabels(ax.get_xticklabels(),rotation=45,horizontalalignment='right')

lower = [line.get_ydata().min() for line in ax.lines]
upper = [line.get_ydata().max() for line in ax.lines]
#means = ax.collections[0].get_offsets()[:, 1]
#means = ax.containers
#means = ax.containers[0].get_offsets()[:, 1]
#means = ax.containers[0].get_datavalues()[:, 1]
means = [rectangle.get_height().max() for rectangle in ax.patches]
labels = [line.get_xdata() for line in ax.lines]
#print(lower)
#print(means)
print(labels)

In [ ]:
# violin plots

In [ ]:
jcoin_json = json.loads(Path("jcoin_states.json").read_text())
jcoin_df = pd.DataFrame(jcoin_json)\
    .assign(hub_types=lambda df:df["hub"]+"("+df["type"]+")")\
    .groupby('states')\
    .agg(
        {"hub_types":lambda s:",".join(s),
        "hub":"count"}
        )\
    .reset_index()

In [ ]:
is_jcoin_state = sub_df_1["state_cd"].isin(jcoin_df.index.values)
sub_df_1["is_jcoin_state"] = np.where(
    sub_df_1["state_cd"].isin(jcoin_df["states"]),
        "Yes","No" 
    )


In [ ]:
fig, ax = plt.subplots()

sns.barplot(
    ax=ax, x="state",
     y="score6_and_weight2", 
     data=sub_df_1.sort_values('is_jcoin_state'), 
     estimator=weighted_mean_2,
     orient='v',
     hue="is_jcoin_state")
#ax.set_ylim(2, 3.55) 

plt.draw()
ax.set_xticks(ax.get_xticks())
ax.set_xticklabels(ax.get_xticklabels(),rotation=45,horizontalalignment='right')

lower = [line.get_ydata().min() for line in ax.lines]
upper = [line.get_ydata().max() for line in ax.lines]
#means = ax.collections[0].get_offsets()[:, 1]
#means = ax.containers
#means = ax.containers[0].get_offsets()[:, 1]
#means = ax.containers[0].get_datavalues()[:, 1]
means = [rectangle.get_height().max() for rectangle in ax.patches]
labels = [line.get_xdata() for line in ax.lines]
#print(lower)
#print(means)
print(labels)

In [ ]:
sub_df_1["score6_and_weight2"] = sub_df_1["score6_and_weight2"].astype(float)

In [ ]:
fig, ax = plt.subplots()

sns.violinplot(
    ax=ax, x="is_jcoin_state",
     y="score6_and_weight2", 
     data=sub_df_1, 
     estimator=weighted_mean_2)
#ax.set_ylim(2, 3.55) 

plt.draw()
ax.set_xticks(ax.get_xticks())
ax.set_xticklabels(ax.get_xticklabels(),rotation=45,horizontalalignment='right')

lower = [line.get_ydata().min() for line in ax.lines]
upper = [line.get_ydata().max() for line in ax.lines]
#means = ax.collections[0].get_offsets()[:, 1]
#means = ax.containers
#means = ax.containers[0].get_offsets()[:, 1]
#means = ax.containers[0].get_datavalues()[:, 1]
means = [rectangle.get_height().max() for rectangle in ax.patches]
labels = [line.get_xdata() for line in ax.lines]
#print(lower)
#print(means)
print(labels)

```{margin} 
**To go to the data/study page on the HEAL Data Platform, follow this link:** my link
```

```{margin} 
**To go to an interactive analytic cloud workspace with the analysis code and data loaded, follow this link:** my link
```

Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Sodales ut eu sem integer vitae justo eget. Pellentesque dignissim enim sit amet venenatis urna cursus. Sed faucibus turpis in eu mi bibendum. Scelerisque felis imperdiet proin fermentum leo. Volutpat est velit egestas dui id ornare arcu. Quis lectus nulla at volutpat diam ut venenatis tellus. Tellus pellentesque eu tincidunt tortor aliquam nulla facilisi cras. Pellentesque adipiscing commodo elit at imperdiet dui. 
<br>

Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Sodales ut eu sem integer vitae justo eget. Pellentesque dignissim enim sit amet venenatis urna cursus. Sed faucibus turpis in eu mi bibendum. Scelerisque felis imperdiet proin fermentum leo. Volutpat est velit egestas dui id ornare arcu. Quis lectus nulla at volutpat diam ut venenatis tellus. Tellus pellentesque eu tincidunt tortor aliquam nulla facilisi cras. Pellentesque adipiscing commodo elit at imperdiet dui. 
<br><br>
In hac habitasse platea dictumst quisque sagittis purus. Libero volutpat sed cras ornare. Sit amet consectetur adipiscing elit pellentesque habitant morbi tristique senectus. Auctor augue mauris augue neque gravida in fermentum et. Amet mattis vulputate enim nulla aliquet porttitor. Proin sed libero enim sed faucibus turpis in eu. Morbi tristique senectus et netus et malesuada. Feugiat sed lectus vestibulum mattis ullamcorper.

**Data Citation** 
<br>
Harold Pollack, Johnathon Schneider, Bruce Taylor. JCOIN 026: Brief Stigma Survey. Chicago, IL: Center for Translational Data Science HEAL Data Platform (distributor) via Center for Translational Data Science JCOIN Data Commons (repository & distributor), 2022-04-08. (HEAL Data Platform branded doi goes here)
<br>
**Brief Article Citation** 
<br>
What format should this be? 